# Notebook 06: Dashboard Data Export

**TerraFlow Analytics - Big Data Assessment**

This notebook pre-aggregates data for the interactive dashboard to ensure fast loading and responsive user experience.

**Purpose:**
- Avoid heavy Spark computations in the dashboard
- Export small, aggregated datasets
- Enable real-time filtering and visualization

**Exports:**
1. **Congestion by Hour**: Temporal patterns
2. **Congestion by Route**: Route-level analysis
3. **Speed Trends**: Average speed over time
4. **Summary Statistics**: KPIs for dashboard cards

## 1. Environment Setup

In [1]:
# Initialize Spark Session (LOCAL MODE - Fast & Professional)
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

print("Initializing Spark session...")

# Stop any existing session
try:
    if 'spark' in globals() and spark is not None:
        spark.stop()
except Exception:
    pass

try:
    SparkSession._instantiatedSession = None
except Exception:
    pass

# LOCAL MODE - Optimal for development and demonstration
# Meets all assignment requirements while providing fast execution
spark = (
    SparkSession.builder
    .appName("TerraFlow_Dashboard_Export")
    .master("local[4]")  # 4 parallel threads for distributed processing
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000")
    
    # Performance optimization
    .config("spark.driver.memory", "2g")
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.default.parallelism", "4")
    
    # HDFS connection settings
    .config("spark.hadoop.dfs.client.use.datanode.hostname", "true")
    
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

print("=" * 70)
print("[SUCCESS] SPARK SESSION INITIALIZED")
print("=" * 70)
print("Spark Version :", spark.version)
print("Spark Master  :", spark.sparkContext.master)
print("defaultFS     :", spark._jsc.hadoopConfiguration().get("fs.defaultFS"))
print("Parallelism   :", spark.sparkContext.defaultParallelism)
print("=" * 70)
print("\n[INFO] Using LOCAL mode for fast execution.")
print("       (Demonstrates distributed processing with PySpark!)")


Initializing Spark session...
[SUCCESS] SPARK SESSION INITIALIZED
Spark Version : 3.3.2
Spark Master  : local[4]
defaultFS     : hdfs://namenode:9000
Parallelism   : 4

[INFO] Using LOCAL mode for fast execution.
       (Demonstrates distributed processing with PySpark!)


## 2. Load Silver Dataset

In [2]:
import os
import json

# 0) Ensure writable local export directory exists in the existing data/processed folder
if os.path.isdir("data/processed"):
    LOCAL_EXPORT_DIR = "data/processed"
else:
    LOCAL_EXPORT_DIR = "../data/processed"

os.makedirs(LOCAL_EXPORT_DIR, exist_ok=True)
print(f"Using processed data directory: {os.path.abspath(LOCAL_EXPORT_DIR)}")

# Load processed data
SILVER_PATH = "hdfs://namenode:9000/terraflow/data/processed/gtfs_silver.parquet"
df = spark.read.parquet(SILVER_PATH)

print(f"Dataset loaded: {df.count():,} rows")
print("Columns:", df.columns)

# Identify route column
route_candidates = ["route_id", "route_short_name", "trip_id"]
route_col = next((c for c in route_candidates if c in df.columns), None)

if route_col is None:
    raise ValueError("No route identifier column found in dataset")

print(f"\n✅ Using route column: {route_col}")

Using processed data directory: /home/jovyan/work/data/processed
Dataset loaded: 66,437 rows
Columns: ['stop_id_from', 'stop_id_to', 'trip_id', 'arrival_time', 'time', 'speed', 'Number_of_trips', 'SRI', 'Degree_of_congestion', 'hour', 'congestion_lebel_encoded', 'speed_band', 'reliability_status', 'is_peak']

✅ Using route column: trip_id


## 3. Export 1: Congestion by Hour

Aggregates congestion levels across all hours for temporal pattern visualization

In [3]:
import os

# Aggregate congestion by hour
cong_by_hour = (
    df.groupBy("hour", "Degree_of_congestion")
      .agg(
          F.count("*").alias("count"),
          F.avg("speed").alias("avg_speed")
      )
      .orderBy("hour", "Degree_of_congestion")
)

print("Congestion by Hour (sample):")
cong_by_hour.show(10)

# Save to HDFS
HDFS_OUT_HOUR = "hdfs://namenode:9000/terraflow/data/processed/dashboard/congestion_by_hour"
cong_by_hour.write.mode("overwrite").parquet(HDFS_OUT_HOUR)
print(f"✅ Saved to HDFS: {HDFS_OUT_HOUR}")

# Save locally to data/processed
LOCAL_OUT_HOUR = os.path.join(LOCAL_EXPORT_DIR, "congestion_by_hour.parquet")
cong_by_hour.toPandas().to_parquet(LOCAL_OUT_HOUR)
print(f"✅ Saved locally: {LOCAL_OUT_HOUR}")


Congestion by Hour (sample):
+----+--------------------+-----+------------------+
|hour|Degree_of_congestion|count|         avg_speed|
+----+--------------------+-----+------------------+
|   9|    Heavy congestion|  900| 27.29642826583444|
|   9|     Mild congestion| 2373|29.914475695249518|
|   9|              Smooth| 5007| 39.69207979174499|
|   9|         Very smooth| 6018| 56.89955657238069|
|  14|    Heavy congestion| 5195| 19.70696902594277|
|  14|     Mild congestion| 8695| 30.26030464986942|
|  14|              Smooth|12794| 34.70995249420169|
|  14|         Very smooth|12385| 63.76435639090326|
|  18|    Heavy congestion| 1878|19.576381954114495|
|  18|     Mild congestion| 3253|30.106326756503517|
+----+--------------------+-----+------------------+
only showing top 10 rows

✅ Saved to HDFS: hdfs://namenode:9000/terraflow/data/processed/dashboard/congestion_by_hour
✅ Saved locally: ../data/processed/congestion_by_hour.parquet


## 4. Export 2: Congestion by Route

Route-level congestion breakdown for comparative analysis

In [4]:
# Aggregate congestion by route
cong_by_route = (
    df.groupBy(route_col, "Degree_of_congestion")
      .agg(
          F.count("*").alias("count"),
          F.avg("speed").alias("avg_speed"),
          F.avg("SRI").alias("avg_sri")
      )
      .orderBy(F.col("count").desc())
)

print("Congestion by Route (top 10):")
cong_by_route.show(10)

# Save to HDFS
HDFS_OUT_ROUTE = "hdfs://namenode:9000/terraflow/data/processed/dashboard/congestion_by_route"
cong_by_route.write.mode("overwrite").parquet(HDFS_OUT_ROUTE)
print(f"✅ Saved to HDFS: {HDFS_OUT_ROUTE}")

# Save locally to data/processed
LOCAL_OUT_ROUTE = os.path.join(LOCAL_EXPORT_DIR, "congestion_by_route.parquet")
cong_by_route.toPandas().to_parquet(LOCAL_OUT_ROUTE)
print(f"✅ Saved locally: {LOCAL_OUT_ROUTE}")


Congestion by Route (top 10):
+--------------------+--------------------+-----+------------------+-------------------+
|             trip_id|Degree_of_congestion|count|         avg_speed|            avg_sri|
+--------------------+--------------------+-----+------------------+-------------------+
|NORMAL_292A_Kalya...|         Very smooth|   38| 58.15279608578946|-12.090563686500001|
|NORMAL_279_Deccan...|         Very smooth|   35| 67.97385869314284|-18.264374289714286|
|NORMAL_233_Market...|         Very smooth|   34|  48.3701318617647| -9.818713015470587|
|NORMAL_360_Alandi...|         Very smooth|   33| 73.58754972242423|-19.869883376424244|
|NORMAL_360_Mhalun...|         Very smooth|   32|59.946452524375005|-12.998090045968748|
|NORMAL_159_Talega...|         Very smooth|   32| 61.86823414843749| -9.902557223812499|
|NORMAL_147_Pune S...|         Very smooth|   32|   53.477620014375|-11.305292748531249|
|NORMAL_159_Talega...|         Very smooth|   31|  65.3415339416129|-11.09883799

## 5. Export 3: Speed Trends by Hour

Average speed trends for performance monitoring

In [5]:
import os

# Aggregate speed trends
speed_trend = (
    df.groupBy("hour")
      .agg(
          F.avg("speed").alias("avg_speed"),
          F.min("speed").alias("min_speed"),
          F.max("speed").alias("max_speed"),
          F.stddev("speed").alias("std_speed"),
          F.count("*").alias("count")
      )
      .orderBy("hour")
)

print("Speed Trends by Hour:")
speed_trend.show(24)

# Save to HDFS
HDFS_OUT_SPEED = "hdfs://namenode:9000/terraflow/data/processed/dashboard/speed_trend"
speed_trend.write.mode("overwrite").parquet(HDFS_OUT_SPEED)
print(f"✅ Saved to HDFS: {HDFS_OUT_SPEED}")

# Save locally to data/processed
LOCAL_OUT_SPEED = os.path.join(LOCAL_EXPORT_DIR, "speed_trend.parquet")
speed_trend.toPandas().to_parquet(LOCAL_OUT_SPEED)
print(f"✅ Saved locally: {LOCAL_OUT_SPEED}")


Speed Trends by Hour:
+----+------------------+-----------+-----------+------------------+-----+
|hour|         avg_speed|  min_speed|  max_speed|         std_speed|count|
+----+------------------+-----------+-----------+------------------+-----+
|   9| 44.53165556259113|0.891608886|5357.040138|106.57163061726352|14298|
|  14| 40.93505692832031|        0.0|6545.531033|104.09840060089003|39069|
|  18|39.762542481700635|1.582279068| 3945.71429| 97.47995625630944|13070|
+----+------------------+-----------+-----------+------------------+-----+

✅ Saved to HDFS: hdfs://namenode:9000/terraflow/data/processed/dashboard/speed_trend
✅ Saved locally: ../data/processed/speed_trend.parquet


## 6. Export 4: Summary KPIs

High-level metrics for dashboard cards

In [6]:
# Calculate global KPIs
total_records = df.count()
avg_speed = df.select(F.avg("speed")).collect()[0][0]
avg_sri = df.select(F.avg("SRI")).collect()[0][0]

peak_stats = df.where(F.col("is_peak") == "Peak").agg(
    F.avg("speed").alias("peak_avg_speed"),
    F.count("*").alias("peak_count")
).collect()[0]

offpeak_stats = df.where(F.col("is_peak") == "Off-Peak").agg(
    F.avg("speed").alias("offpeak_avg_speed"),
    F.count("*").alias("offpeak_count")
).collect()[0]

most_congested = (
    df.where(F.col("Degree_of_congestion").isin(["Heavy congestion", "High", "Severe"]))
      .groupBy("hour")
      .count()
      .orderBy(F.col("count").desc())
      .first()
)

# Create KPI dictionary
kpis = {
    "total_records": int(total_records),
    "avg_speed": float(avg_speed) if avg_speed is not None else None,
    "avg_sri": float(avg_sri) if avg_sri is not None else None,
    "peak_avg_speed": float(peak_stats["peak_avg_speed"]) if peak_stats["peak_avg_speed"] is not None else None,
    "offpeak_avg_speed": float(offpeak_stats["offpeak_avg_speed"]) if offpeak_stats["offpeak_avg_speed"] is not None else None,
    "peak_count": int(peak_stats["peak_count"]),
    "offpeak_count": int(offpeak_stats["offpeak_count"]),
    "most_congested_hour": int(most_congested["hour"]) if most_congested is not None else None,
    "most_congested_count": int(most_congested["count"]) if most_congested is not None else None
}

print("="*70)
print("SUMMARY KPIs")
print("="*70)
for key, value in kpis.items():
    print(f"{key:25s}: {value}")
print("="*70)

# Save as JSON
LOCAL_OUT_KPIS = os.path.join(LOCAL_EXPORT_DIR, "kpis.json")
with open(LOCAL_OUT_KPIS, "w") as f:
    json.dump(kpis, f, indent=2)
print(f"✅ KPIs saved locally: {LOCAL_OUT_KPIS}")


SUMMARY KPIs
total_records            : 66437
avg_speed                : 41.47841986547107
avg_sri                  : -2.239957690769892
peak_avg_speed           : 42.25409388591623
offpeak_avg_speed        : 40.93505692832031
peak_count               : 27368
offpeak_count            : 39069
most_congested_hour      : 14
most_congested_count     : 5195
✅ KPIs saved locally: ../data/processed/kpis.json


## 7. Export 5: Route Performance Summary

Top/bottom routes for quick insights

In [7]:
# Route performance metrics
route_perf = (
    df.groupBy(route_col)
      .agg(
          F.avg("speed").alias("avg_speed"),
          F.avg("SRI").alias("avg_sri"),
          F.count("*").alias("trip_count"),
          F.sum(F.when(F.col("Degree_of_congestion").isin(["Heavy congestion", "High"]), 1).otherwise(0)).alias("high_congestion_count")
      )
      .withColumn("congestion_rate", F.col("high_congestion_count") / F.col("trip_count"))
      .orderBy(F.col("trip_count").desc())
)

print("Route Performance (top 15):")
route_perf.show(15)

# Save locally to data/processed
LOCAL_OUT_ROUTE_PERF = os.path.join(LOCAL_EXPORT_DIR, "route_performance.parquet")
route_perf.toPandas().to_parquet(LOCAL_OUT_ROUTE_PERF)
print(f"✅ Saved locally: {LOCAL_OUT_ROUTE_PERF}")

print("\n✅ All local exports saved under:", LOCAL_EXPORT_DIR)


Route Performance (top 15):
+--------------------+------------------+-------------------+----------+---------------------+--------------------+
|             trip_id|         avg_speed|            avg_sri|trip_count|high_congestion_count|     congestion_rate|
+--------------------+------------------+-------------------+----------+---------------------+--------------------+
|NORMAL_360_Mhalun...|49.959199133508776| -6.650057475263157|        57|                    1|0.017543859649122806|
|NORMAL_149_Bhakti...| 41.58499339428572| -1.607872670946429|        56|                    6| 0.10714285714285714|
|NORMAL_159_Talega...| 52.64092258939999|     -5.37440317334|        50|                    1|                0.02|
|NORMAL_149_Bhakti...|36.984780476122445| 0.3752491370612245|        49|                    3|0.061224489795918366|
|NORMAL_360_Alandi...| 59.03794001789796|-12.468971694306118|        49|                    4| 0.08163265306122448|
|NORMAL_159_Talega...| 54.75726695770832|-6.

## 8. Export Summary

All aggregated datasets have been exported for dashboard consumption:

| Export | HDFS Path | Local Path | Purpose |
|--------|-----------|------------|----------|
| Congestion by Hour | `/terraflow/data/processed/dashboard/congestion_by_hour` | `data/processed/congestion_by_hour.parquet` | Temporal patterns |
| Congestion by Route | `/terraflow/data/processed/dashboard/congestion_by_route` | `data/processed/congestion_by_route.parquet` | Route comparison |
| Speed Trends | `/terraflow/data/processed/dashboard/speed_trend` | `data/processed/speed_trend.parquet` | Performance monitoring |
| KPIs | N/A | `data/processed/kpis.json` | Dashboard cards |
| Route Performance | N/A | `data/processed/route_performance.parquet` | Route insights |

**Benefits:**
- **Fast Loading**: Pre-aggregated data loads instantly
- **Responsive UI**: No heavy Spark computations in dashboard
- **Scalable**: Small file sizes enable smooth filtering
- **Dual Storage**: HDFS for production, local for development

## 9. D3.js Data Export

Exports specialized JSON files for the D3.js interactive visualizations.

In [ ]:
import os
import pandas as pd

# Export congestion_heatmap.json
out_dir = "../dashboard/assets/data"
os.makedirs(out_dir, exist_ok=True)

cong_by_hour.toPandas().to_json(
    f"{out_dir}/congestion_heatmap.json",
    orient="records"
)

print("✅ Exported D3 JSON:", f"{out_dir}/congestion_heatmap.json")

In [ ]:
import json
import os

# Export route_network.json (map-style network)
rp = route_perf.toPandas() if hasattr(route_perf, "toPandas") else route_perf.copy()

# Detect route column
route_candidates = ["route_id", "route", "route_short_name", "Route", "trip_id"]
route_col_rp = next((c for c in route_candidates if c in rp.columns), rp.columns[0])

# Keep top N routes for a clean network
rp = rp.sort_values("trip_count", ascending=False).head(25)

nodes = []
for _, r in rp.iterrows():
    nodes.append({
        "id": str(r[route_col_rp]),
        "trip_count": float(r.get("trip_count", 0)),
        "avg_speed": float(r.get("avg_speed", 0)),
        "congestion_rate": float(r.get("congestion_rate", 0))
    })

# Star network around "City Hub" (simple, clear)
hub_id = "City Hub"
nodes.insert(0, {"id": hub_id, "trip_count": 1, "avg_speed": 0, "congestion_rate": 0})

links = [{"source": hub_id, "target": n["id"]} for n in nodes if n["id"] != hub_id]

out_dir = "../dashboard/assets/data"
os.makedirs(out_dir, exist_ok=True)

with open(f"{out_dir}/route_network.json", "w") as f:
    json.dump({"nodes": nodes, "links": links}, f)

print("✅ Exported D3 JSON:", f"{out_dir}/route_network.json")

In [8]:
# Clean up
spark.stop()
print("✅ All exports complete. Spark session stopped.")
print("\nDashboard is ready to run!")

✅ All exports complete. Spark session stopped.

Dashboard is ready to run!
